## Vector query

### Create embedding model

In [3]:

## Todo: 
# Change endpoint_name to your Sagemaker sbert deployed model.

## Langchain Doc for SM Embedding Class Customization

from typing import Dict, List
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
import json


class ContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        """
        Transforms the input into bytes that can be consumed by SageMaker endpoint.
        Args:
            inputs: List of input strings.
            model_kwargs: Additional keyword arguments to be passed to the endpoint.
        Returns:
            The transformed bytes input.
        """
        # Example: inference.py expects a JSON string with a "inputs" key:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})  
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        """
        Transforms the bytes output from the endpoint into a list of embeddings.
        Args:
            output: The bytes output from SageMaker endpoint.
        Returns:
            The transformed output - list of embeddings
        Note:
            The length of the outer list is the number of input strings.
            The length of the inner lists is the embedding dimension.
        """
        # Example: inference.py returns a JSON string with the list of
        # embeddings in a "vectors" key:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["vectors"]


content_handler = ContentHandler()



sbert_batch_embeddings = SagemakerEndpointEmbeddings(
    # credentials_profile_name="credentials-profile-name",
    endpoint_name="huggingface-pytorch-inference-2023-10-20-04-45-11-397", # change this to your own sbert endpoint
    region_name="us-east-1",  # change this to your sagemaker deployed sbert endpoint Region 
    content_handler=content_handler,
)

### Call AOS which was created before

In [12]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from langchain.vectorstores import OpenSearchVectorSearch
import boto3
import langchain 


service = 'aoss'
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, "us-east-1", service)   # TODO: change your region

index_name = "rag-sbert"

docsearch = OpenSearchVectorSearch(
    "https://1n3li4pv4s7jhgykpmie.us-east-1.aoss.amazonaws.com:443",  # TODO: Change your AOS endpoint here
    index_name,
    sbert_batch_embeddings,
    http_auth=auth,
    timeout = 100,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    engine="faiss"
)

### Query relative docs given a question (query)

In [14]:
from langchain.load.dump import dumps


query = "Nội dung chính là gì?"

results = docsearch.similarity_search(query, k=3)  # our search query  # return 3 most relevant docs

print(dumps(results, pretty=True))


[
  {
    "lc": 1,
    "type": "constructor",
    "id": [
      "langchain",
      "schema",
      "document",
      "Document"
    ],
    "kwargs": {
      "page_content": "T\u1ea1o qu\u1ea3ng c\u00e1o s\u00e1ng t\u1ea1o b\u1eb1ng c\u00e1ch s\u1eed d\u1ee5ng AI t\u1ea1o \u0111\u01b0\u1ee3c tri\u1ec3n khai tr\u00ean Amazon SageMaker Qu\u1ea3ng c\u00e1o s\u00e1ng t\u1ea1o c\u00f3 ti\u1ec1m n\u0103ng \u0111\u01b0\u1ee3c c\u00e1ch m\u1ea1ng h\u00f3a b\u1edfi AI t\u1ea1o (GenAI). B\u00e2y gi\u1edd b\u1ea1n c\u00f3 th\u1ec3 t\u1ea1o ra m\u1ed9t lo\u1ea1t c\u00e1c h\u00ecnh \u1ea3nh m\u1edbi, ch\u1eb3ng h\u1ea1n nh\u01b0 \u1ea3nh s\u1ea3n ph\u1ea9m, b\u1eb1ng c\u00e1ch \u0111\u00e0o t\u1ea1o l\u1ea1i m\u00f4 h\u00ecnh GenAI v\u00e0 cung c\u1ea5p m\u1ed9t v\u00e0i \u0111\u1ea7u v\u00e0o m\u00f4 h\u00ecnh, ch\u1eb3ng h\u1ea1n nh\u01b0 l\u1eddi nh\u1eafc v\u0103n b\u1ea3n (nh\u1eefng c\u00e2u m\u00f4 t\u1ea3 c\u1ea3nh v\u00e0 c\u00e1c \u0111\u1ed1i t\u01b0\u1ee3ng \u0111\u01b0\u1ee3c m\u00f4 h\

### Question Answering

#### Bedrock client config

In [17]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
"""Helper utilities for working with Amazon Bedrock from Python notebooks"""
# Python Built-Ins:
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides

    Parameters
    ----------
    assumed_role :
        Optional ARN of an AWS IAM role to assume for calling the Bedrock service. If not
        specified, the current active credentials will be used.
    region :
        Optional name of the AWS Region in which the service should be called (e.g. "us-east-1").
        If not specified, AWS_REGION or AWS_DEFAULT_REGION environment variable will be used.
    runtime :
        Optional choice of getting different client to perform operations with the Amazon Bedrock service.
    """
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client



from langchain.llms.bedrock import Bedrock
from langchain.load.dump import dumps



os.environ["AWS_DEFAULT_REGION"] = "us-east-1" 

boto3_bedrock = get_bedrock_client(
    #assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

# - create the Anthropic Model
llm = Bedrock(
    model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 200}
)



Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


#### QnA 

In [18]:

from langchain.chains import RetrievalQA



qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


In [19]:
qa.run("Nội dung chính là gì?")


' Dựa trên nội dung đã cung cấp, một số ý chính của bài viết này là:\n\n- Sử dụng công nghệ AI tạo (GenAI) để tạo ra quảng cáo sáng tạo với hình ảnh sản phẩm mới dựa trên lời nhắc văn bản. Các mô hình GenAI như Stable Diffusion và Dall-E 2 cho thấy tiềm năng lớn.\n\n- Đề xuất sử dụng'

#### QnA with citation (sources)

In [20]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={'k': 3}),return_source_documents=True)

In [22]:
print(dumps(qa_with_sources(query), pretty=True)) ## NOTE: The answer is in UNICODE8, so we need to convert to Vietnamese

{
  "query": "N\u1ed9i dung ch\u00ednh l\u00e0 g\u00ec?",
  "result": " T\u1eeb nh\u1eefng th\u00f4ng tin \u0111\u01b0\u1ee3c cung c\u1ea5p, ta c\u00f3 th\u1ec3 r\u00fat ra c\u00e1c n\u1ed9i dung ch\u00ednh nh\u01b0 sau:\n\n- B\u00e0i vi\u1ebft kh\u00e1m ph\u00e1 c\u00e1ch c\u00f4ng ngh\u1ec7 AI trong vi\u1ec7c t\u1ea1o h\u00ecnh \u1ea3nh (GenAI) c\u00f3 th\u1ec3 \u0111\u01b0\u1ee3c \u1ee9ng d\u1ee5ng \u0111\u1ec3 t\u1ea1o ra qu\u1ea3ng c\u00e1o s\u00e1ng t\u1ea1o h\u01a1n. \u0110\u1eb7c bi\u1ec7t th\u00f4ng qua k\u1ef9 thu\u1eadt v\u1ebd/s\u01a1n, c\u00f3 th\u1ec3 t\u1ea1o ra h\u00ecnh \u1ea3nh li\u1ec1n m\u1ea1ch, h\u1ea5p d",
  "source_documents": [
    {
      "lc": 1,
      "type": "constructor",
      "id": [
        "langchain",
        "schema",
        "document",
        "Document"
      ],
      "kwargs": {
        "page_content": "T\u1ea1o qu\u1ea3ng c\u00e1o s\u00e1ng t\u1ea1o b\u1eb1ng c\u00e1ch s\u1eed d\u1ee5ng AI t\u1ea1o \u0111\u01b0\u1ee3c tri\u1ec3n khai tr\u00ean A

In [33]:
text = b'T\u1eeb nh\u1eefng th\u00f4ng tin \u0111\u01b0\u1ee3c cung c\u1ea5p, ta c\u00f3 th\u1ec3 r\u00fat ra c\u00e1c n\u1ed9i dung ch\u00ednh nh\u01b0 sau:\n\n- B\u00e0i vi\u1ebft kh\u00e1m ph\u00e1 c\u00e1ch c\u00f4ng ngh\u1ec7 AI trong vi\u1ec7c t\u1ea1o h\u00ecnh \u1ea3nh (GenAI) c\u00f3 th\u1ec3 \u0111\u01b0\u1ee3c \u1ee9ng d\u1ee5ng \u0111\u1ec3 t\u1ea1o ra qu\u1ea3ng c\u00e1o s\u00e1ng t\u1ea1o h\u01a1n. \u0110\u1eb7c bi\u1ec7t th\u00f4ng qua k\u1ef9 thu\u1eadt v\u1ebd/s\u01a1n, c\u00f3 th\u1ec3 t\u1ea1o ra h\u00ecnh \u1ea3nh li\u1ec1n m\u1ea1ch, h\u1ea5p d'

In [35]:
utf8_text = b'T\u1eeb nh\u1eefng th\u00f4ng tin \u0111\u01b0\u1ee3c cung c\u1ea5p, ta c\u00f3 th\u1ec3 r\u00fat ra c\u00e1c n\u1ed9i dung ch\u00ednh nh\u01b0 sau:\n\n- B\u00e0i vi\u1ebft kh\u00e1m ph\u00e1 c\u00e1ch c\u00f4ng ngh\u1ec7 AI trong vi\u1ec7c t\u1ea1o h\u00ecnh \u1ea3nh (GenAI) c\u00f3 th\u1ec3 \u0111\u01b0\u1ee3c \u1ee9ng d\u1ee5ng \u0111\u1ec3 t\u1ea1o ra qu\u1ea3ng c\u00e1o s\u00e1ng t\u1ea1o h\u01a1n. \u0110\u1eb7c bi\u1ec7t th\u00f4ng qua k\u1ef9 thu\u1eadt v\u1ebd/s\u01a1n, c\u00f3 th\u1ec3 t\u1ea1o ra h\u00ecnh \u1ea3nh li\u1ec1n m\u1ea1ch, h\u1ea5p d'

# Decode UTF-8 bytes to Unicode
unicode_text = utf8_text.decode('utf-8') 
unicode_text

'T\\u1eeb nh\\u1eefng th\\u00f4ng tin \\u0111\\u01b0\\u1ee3c cung c\\u1ea5p, ta c\\u00f3 th\\u1ec3 r\\u00fat ra c\\u00e1c n\\u1ed9i dung ch\\u00ednh nh\\u01b0 sau:\n\n- B\\u00e0i vi\\u1ebft kh\\u00e1m ph\\u00e1 c\\u00e1ch c\\u00f4ng ngh\\u1ec7 AI trong vi\\u1ec7c t\\u1ea1o h\\u00ecnh \\u1ea3nh (GenAI) c\\u00f3 th\\u1ec3 \\u0111\\u01b0\\u1ee3c \\u1ee9ng d\\u1ee5ng \\u0111\\u1ec3 t\\u1ea1o ra qu\\u1ea3ng c\\u00e1o s\\u00e1ng t\\u1ea1o h\\u01a1n. \\u0110\\u1eb7c bi\\u1ec7t th\\u00f4ng qua k\\u1ef9 thu\\u1eadt v\\u1ebd/s\\u01a1n, c\\u00f3 th\\u1ec3 t\\u1ea1o ra h\\u00ecnh \\u1ea3nh li\\u1ec1n m\\u1ea1ch, h\\u1ea5p d'

In [36]:
import unicodedata

text = r"T\u1eeb nh\u1eefng th\u00f4ng tin \u0111\u01b0\u1ee3c cung c\u1ea5p, ta c\u00f3 th\u1ec3 r\u00fat ra c\u00e1c n\u1ed9i dung ch\u00ednh nh\u01b0 sau:\n\n- B\u00e0i vi\u1ebft kh\u00e1m ph\u00e1 c\u00e1ch c\u00f4ng ngh\u1ec7 AI trong vi\u1ec7c t\u1ea1o h\u00ecnh \u1ea3nh (GenAI) c\u00f3 th\u1ec3 \u0111\u01b0\u1ee3c \u1ee9ng d\u1ee5ng \u0111\u1ec3 t\u1ea1o ra qu\u1ea3ng c\u00e1o s\u00e1ng t\u1ea1o h\u01a1n. \u0110\u1eb7c bi\u1ec7t th\u00f4ng qua k\u1ef9 thu\u1eadt v\u1ebd/s\u01a1n, c\u00f3 th\u1ec3 t\u1ea1o ra h\u00ecnh \u1ea3nh li\u1ec1n m\u1ea1ch, h\u1ea5p d"

# Normalize Unicode  
normalized_text = unicodedata.normalize('NFC', text)

# Encode as Vietnamese UTF-8
vietnamese_text = normalized_text.encode('utf-8')

print(vietnamese_text.decode('utf-8'))

T\u1eeb nh\u1eefng th\u00f4ng tin \u0111\u01b0\u1ee3c cung c\u1ea5p, ta c\u00f3 th\u1ec3 r\u00fat ra c\u00e1c n\u1ed9i dung ch\u00ednh nh\u01b0 sau:\n\n- B\u00e0i vi\u1ebft kh\u00e1m ph\u00e1 c\u00e1ch c\u00f4ng ngh\u1ec7 AI trong vi\u1ec7c t\u1ea1o h\u00ecnh \u1ea3nh (GenAI) c\u00f3 th\u1ec3 \u0111\u01b0\u1ee3c \u1ee9ng d\u1ee5ng \u0111\u1ec3 t\u1ea1o ra qu\u1ea3ng c\u00e1o s\u00e1ng t\u1ea1o h\u01a1n. \u0110\u1eb7c bi\u1ec7t th\u00f4ng qua k\u1ef9 thu\u1eadt v\u1ebd/s\u01a1n, c\u00f3 th\u1ec3 t\u1ea1o ra h\u00ecnh \u1ea3nh li\u1ec1n m\u1ea1ch, h\u1ea5p d
